# Federal Election Results 2018
### Non-Presidential year

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

import psycopg2
from sqlalchemy import create_engine

In [2]:
"""
https://www.fec.gov/introduction-campaign-finance/election-and-voting-information/federal-elections-2018/
Extract multiple sheets from the Excel file and load them into Pandas data frames. 
A dictionary allows you to store each data frame under a unique key (the sheet name), 
making it easy to access and work with each data frame later.
"""
# Define the file path to the Excel file
file_path = 'Election_Results/federalelections2018.xlsx'

# Define a list of sheet names to extract
sheet_names = ['2018 US Senate Results by State', 
               '2018 US House Results by State']

# Create an empty dictionary to store the data frames
data_frames = {}

# Loop over the sheet names and read them into data frames
for name in sheet_names:
    data_frames[name] = pd.read_excel(file_path, sheet_name=name)

# Print the data frames to verify that they were loaded correctly
# for name, df in data_frames.items():
#     print(f"{name}:")
#     print(df.head())


2018 US House Results by State
2018 US House Results by State


In [3]:
df_senate_2018 = data_frames['2018 US Senate Results by State']
df_senate_2018.head(20)
# 681 rows 18 columns

,1,STATE ABBREVIATION,STATE,DISTRICT,FEC ID#,(I),CANDIDATE NAME (First),CANDIDATE NAME (Last),CANDIDATE NAME,TOTAL VOTES,...,RUNOFF VOTES,RUNOFF %,GENERAL VOTES,GENERAL %,GE RUNOFF ELECTION VOTES (MS Senate),GE RUNOFF ELECTION % (MS Senate),"COMBINED GE PARTY TOTALS (CT, NY)","COMBINED % (CT, NY)",GE WINNER INDICATOR,FOOTNOTES
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,AZ,Arizona,S,S8AZ00197,NaN,Kyrsten,Sinema,"Sinema, Kyrsten",NaN,...,NaN,NaN,1191100.0,4.995579e-01,NaN,NaN,NaN,NaN,W,NaN
2,4,AZ,Arizona,S,S8AZ00122,NaN,Deedra,Abboud,"Abboud, Deedra",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,AZ,Arizona,S,NaN,NaN,NaN,NaN,NaN,Party Votes:,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,AZ,Arizona,S,S8AZ00221,NaN,Martha,McSally,"McSally, Martha",NaN,...,NaN,NaN,1135200.0,4.761130e-01,NaN,NaN,NaN,NaN,NaN,NaN
5,7,AZ,Arizona,S,S6AZ00233,NaN,Kelli,Ward,"Ward, Kelli",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,AZ,Arizona,S,S8AZ00247,NaN,Joe,Arpaio,"Arpaio, Joe",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,AZ,Arizona,S,NaN,NaN,Nicholas N.,Glenn,"Glenn, Nicholas N.",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,AZ,Arizona,S,NaN,NaN,William,Gonzales,"Gonzales, William",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,AZ,Arizona,S,NaN,NaN,NaN,NaN,NaN,Party Votes:,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# select the columns wanted and rename them
df_selection = df_senate_2018[['STATE ABBREVIATION', 'DISTRICT', 'CANDIDATE NAME (Last)', 'PARTY', 'GENERAL %', 'GENERAL VOTES ']]
df_senate_2018 = df_selection.rename(columns={
    'DISTRICT': 'Office',
    'STATE ABBREVIATION': 'State',
    'CANDIDATE NAME (Last)': 'Last Name',
    'PARTY': 'Party',
    'GENERAL %': 'Vote %',
    'GENERAL VOTES ': 'Vote count'
})
df_senate_2018.head()
# gives 681 rows

,State,Office,Last Name,Party,Vote %,Vote count
0,NaN,NaN,NaN,NaN,NaN,NaN
1,AZ,S,Sinema,D,0.499558,1191100.0
2,AZ,S,Abboud,D,NaN,NaN
3,AZ,S,NaN,D,NaN,NaN
4,AZ,S,McSally,R,0.476113,1135200.0


In [5]:
# add year column
df_senate_2018['Year'] = 2018

# Arrange columns: Move the Year and Office columns to the front
new_cols = ['Year', 'Office', 'State', 'Last Name', 'Party', 'Vote %', 'Vote count']
df_senate_2018 = df_senate_2018.reindex(columns=new_cols)

# Include only Candidates R, D, IND (independent)
df_senate_2018 = df_senate_2018[df_senate_2018['Party'].isin(['R', 'D', 'IND'])]
df_senate_2018['Party'] = df_senate_2018['Party'].replace('R', 'REP')
df_senate_2018['Party'] = df_senate_2018['Party'].replace('D', 'DEM')

df_senate_2018.head()
# gives 135 rows

,Year,Office,State,Last Name,Party,Vote %,Vote count
1,2018,S,AZ,Sinema,DEM,0.499558,1191100.0
2,2018,S,AZ,Abboud,DEM,NaN,NaN
3,2018,S,AZ,NaN,DEM,NaN,NaN
4,2018,S,AZ,McSally,REP,0.476113,1135200.0
5,2018,S,AZ,Ward,REP,NaN,NaN


In [6]:
df_senate_2018['Office'] = df_senate_2018['Office'].replace('S', 'Senate')
df_senate_2018

,Year,Office,State,Last Name,Party,Vote %,Vote count
1,2018,Senate,AZ,Sinema,DEM,0.499558,1191100.0
2,2018,Senate,AZ,Abboud,DEM,NaN,NaN
3,2018,Senate,AZ,NaN,DEM,NaN,NaN
4,2018,Senate,AZ,McSally,REP,0.476113,1135200.0
5,2018,Senate,AZ,Ward,REP,NaN,NaN
...,...,...,...,...,...,...,...
618,2018,Senate,WY,De La Fuente,REP,NaN,NaN
619,2018,Senate,WY,Van Risseghem,REP,NaN,NaN
621,2018,Senate,WY,NaN,REP,NaN,NaN
622,2018,Senate,WY,Trauner,DEM,0.300988,61227.0


In [7]:
# Remove any unseen NaN's from num rows.
df_s_2018 = df_senate_2018.dropna(subset=['Vote %'])
df_s_2018 = df_s_2018.dropna(subset=['Vote count'])
# confirms NaN's are gone
print(df_s_2018[df_s_2018['Vote %'].isna()])
print(df_s_2018[df_s_2018['Vote count'].isna()])

Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []
Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []


In [9]:
# replace non-finite values with -1
df_s_2018['Vote count'] = df_s_2018['Vote count'].replace([np.nan, np.inf, -np.inf], -1)

# convert to integer
df_s_2018['Vote count'] = df_s_2018['Vote count'].astype(int)
# check data types
print(df_s_2018.dtypes)

Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count      int32
dtype: object


In [10]:
df_s_2018.shape
# gives 79 rows.

(79, 7)

In [11]:
df_s_2018.head(5)

,Year,Office,State,Last Name,Party,Vote %,Vote count
1,2018,Senate,AZ,Sinema,DEM,0.499558,1191100
4,2018,Senate,AZ,McSally,REP,0.476113,1135200
23,2018,Senate,CA,Feinstein,DEM,0.541638,6019422
24,2018,Senate,CA,De Leon,DEM,0.458362,5093942
61,2018,Senate,CT,Murphy,DEM,0.567971,787685


## 2018 House Data

In [12]:
df_house_2018 = data_frames['2018 US House Results by State']
df_house_2018.head()

,1,STATE ABBREVIATION,STATE,DISTRICT,FEC ID#,(I),CANDIDATE NAME (First),CANDIDATE NAME (Last),CANDIDATE NAME,TOTAL VOTES,...,RUNOFF VOTES,RUNOFF %,GENERAL VOTES,GENERAL %,GE RUNOFF ELECTION VOTES (MS Senate),GE RUNOFF ELECTION % (MS Senate),"COMBINED GE PARTY TOTALS (CT, NY)","COMBINED % (CT, NY)",GE WINNER INDICATOR,FOOTNOTES
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,AL,Alabama,01,H4AL01123,(I),Bradley,Byrne,"Byrne, Bradley",NaN,...,NaN,NaN,153228,0.631563,NaN,NaN,NaN,NaN,W,NaN
2,4,AL,Alabama,01,H8AL01066,NaN,"Robert, Jr.",Kennedy,"Kennedy, Robert, Jr.",NaN,...,NaN,NaN,89226,0.367765,NaN,NaN,NaN,NaN,NaN,NaN
3,5,AL,Alabama,01,H8AL01082,NaN,Lizzetta Hill,McConnell,"McConnell, Lizzetta Hill",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,AL,Alabama,01,NaN,NaN,NaN,NaN,NaN,Party Votes:,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# select the columns wanted and rename them
df_sel_house = df_house_2018[['STATE ABBREVIATION', 
                                'CANDIDATE NAME (Last)', 
                                'PARTY', 'GENERAL %', 
                                'GENERAL VOTES ']]

df_hou_2018 = df_sel_house.rename(columns={
    'STATE ABBREVIATION': 'State',
    'CANDIDATE NAME (Last)': 'Last Name',
    'PARTY': 'Party',
    'GENERAL %': 'Vote %',
    'GENERAL VOTES ': 'Vote count'
})
df_hou_2018.head()
# gives 541 rows.

,State,Last Name,Party,Vote %,Vote count
0,NaN,NaN,NaN,NaN,NaN
1,AL,Byrne,R,0.631563,153228
2,AL,Kennedy,D,0.367765,89226
3,AL,McConnell,D,NaN,NaN
4,AL,NaN,D,NaN,NaN


In [14]:
# add year and office columns
df_hou_2018['Year'] = 2018
df_hou_2018['Office'] = 'House'

# Move the Year and Office columns to the front
new_cols = ['Year', 'Office', 'State', 'Last Name', 'Party', 'Vote %', 'Vote count']
df_hou_2018 = df_hou_2018.reindex(columns=new_cols)

# Include only Candidates R, D, IND (independent)
df_hou_2018 = df_hou_2018[df_hou_2018['Party'].isin(['R', 'D', 'IND'])]
df_hou_2018['Party'] = df_hou_2018['Party'].replace('R', 'REP')
df_hou_2018['Party'] = df_hou_2018['Party'].replace('D', 'DEM')

df_hou_2018.head()

,Year,Office,State,Last Name,Party,Vote %,Vote count
1,2018,House,AL,Byrne,REP,0.631563,153228
2,2018,House,AL,Kennedy,DEM,0.367765,89226
3,2018,House,AL,McConnell,DEM,NaN,NaN
4,2018,House,AL,NaN,DEM,NaN,NaN
8,2018,House,AL,Roby,REP,0.613884,138879


In [15]:
# Remove any unseen NaN's from num rows.
df_h_2018 = df_hou_2018.dropna(subset=['Vote %'])
df_h_2018 = df_h_2018.dropna(subset=['Vote count'])
# confirms NaN's are gone
print(df_h_2018[df_h_2018['Vote %'].isna()])
print(df_h_2018[df_h_2018['Vote count'].isna()])

Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []
Empty DataFrame
Columns: [Year, Office, State, Last Name, Party, Vote %, Vote count]
Index: []


In [16]:
df_h_2018.dtypes

Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count     object
dtype: object

In [19]:
# # replace non-finite values with -1
# df_h_2018['Vote count'] = df_h_2018['Vote count'].replace([np.nan, np.inf, -np.inf], -1)

# # convert to integer
# df_h_2018['Vote count'] = df_h_2018['Vote count'].astype(int)
# # check data types
# print(df_h_2018.dtypes)

In [18]:
# replace non-finite values with -1
df_h_2018['Vote count'] = df_h_2018['Vote count'].replace([np.nan, np.inf, -np.inf], -1)

# replace 'Unopposed' with 0
df_h_2018['Vote count'] = df_h_2018['Vote count'].replace('Unopposed', 0)

# convert to integer
df_h_2018['Vote count'] = df_h_2018['Vote count'].astype(int)

# check data types
print(df_h_2018.dtypes)


Year            int64
Office         object
State          object
Last Name      object
Party          object
Vote %        float64
Vote count      int32
dtype: object


## Add data to database

In [20]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database using the results table
# df_p_2020.to_sql("results", engine, if_exists="replace", index=False)
# print("Presidential table loaded successfully")
df_s_2018.to_sql("results", engine, if_exists="append", index=False)
df_h_2018.to_sql("results", engine, if_exists="append", index=False)

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


In [21]:
# import psycopg2
# import pandas as pd

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM results", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()
# gives 1128 rows for 2020
# gives 2084 total row for 2020-2018

      Year Office State  Last Name Party    Vote %  Vote count
0     2020   Pres    AL      Trump   REP  0.620316     1441170
1     2020   Pres    AL      Biden   DEM  0.365700      849624
2     2020   Pres    AL  Jorgensen   IND  0.010836       25176
3     2020   Pres    AK      Trump   REP  0.528331      189951
4     2020   Pres    AK      Biden   DEM  0.427720      153778
...    ...    ...   ...        ...   ...       ...         ...
2079  2018  House    WI   Driessen   IND  0.013679        4416
2080  2018  House    WI  Gallagher   REP  0.636942      209410
2081  2018  House    WI   Liegeois   DEM  0.362757      119265
2082  2018  House    WY     Cheney   REP  0.635857      127963
2083  2018  House    WY     Hunter   DEM  0.297662       59903

[2084 rows x 7 columns]
